# Differential Equations Practicum



Variant 25.

Original Equation:

$$
y' = xy^2-3xy,\:y(0)=2\\
\dfrac{y'}{y^2}=x-\dfrac{3x}{y}\\
z=1/y,\:dz=-\dfrac{dy}{y^2}\\
-z'=x(1-3z)\\
\dfrac{z'}{3z-1}=x\\
\dfrac{\ln{(3z-1)}}{3}=\dfrac{x^2}{2}+C_1\\
-1+3z=C_2e^{\dfrac{3x^2}{2}}\\
\dfrac{3}{y}=C_2e^{\frac{3x^2}{2}}+1\\
y=\dfrac{3}{1+C_2e^{\frac{3x^2}{2}}}\\
For\:y(0)=2\:=>\:2=\dfrac{3}{C_2+1}\:=>\:C_2=\frac{1}{2}
$$

Exact Solution:

$$
y=\frac{6}{2+e^{\frac{3x^2}{2}}}
$$



In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook
import numpy
output_notebook()

Loading BokehJS ...

## Definition of original and exact solution functions

In [2]:
def my_func(x, y):
    return x * y * y - 3 * x * y

In [3]:
def exact_func(x,C):
    return 3/(C*numpy.exp(3*x*x/2)+1)

## Helper function
Get array of xs from xs to xf with N number of times

Created a separate function for it since use it a lot

In [4]:
def get_x(xs, xf, N):
    return numpy.linspace(xs, xf, num=N)

# Computation for the exact function
Receive initial value for x and y, end of the interval [x0,X] and number of steps to perform

Returns x and y arrays

In [5]:
def exact(x0, y0, X, N_steps):
    # F
    func = exact_func
    x = get_x(x0, X, N_steps)
    y = numpy.array(y0)
    # Compute the constant of the general solution
    C = (3/y0-1)/numpy.exp(3*x0*x0/2)
    
    # Compute ys and store them into the array
    # Loop until 50 elements (from 1 because we start with y0)
    for i in range(1, N_steps):
        y = numpy.append(y, func(x[i],C))
    return x, y

## Euler Method
Receive function to compute, initial value for x and y, end of the interval [x0,X] and number of steps to perform

Returns x and y arrays

In [6]:
def euler(func, x0, y0, X, N_steps):
    # Resulting pairs of x, y will be stored in arrays
    x = get_x(x0, X, N_steps)
    y = numpy.array([y0])
    
    # Calculate the step for x
    h = (X - x0) / float(N_steps)
    
    # Loop until 50 elements (-1 because we start with y0)
    for i in range(N_steps-1):
        # Perform Euler method
        # y(i+1)=yi+h*f(xi,yi)
        y = numpy.append(y, y[i] + h * func(x[i], y[i]))
    return x, y

## Improved Euler Method
Receive function to comoute, initial value for x and y, end of the interval [x0,X] and number of steps to perform

Returns x and y arrays

In [7]:
def improved_euler(func, x0, y0, X, N_steps):
    # Resulting pairs of x, y will be stored in arrays
    x = get_x(x0, X, N_steps)
    y = numpy.array([y0])
    
    # Calculate the step for x
    h = (X - x0) / float(N_steps)

    # Loop until 50 elements (-1 because we start with y0)
    for i in range(N_steps-1):
        # Perform Improved Euler Method
        # y(i+1) = yi + h/2 * (f(xi,yi)+f(x(i+1),yi+h*f(xi,yi)))
        # where m1 = f(xi,yi) and m2 = f(x(i+1),yi+h*m1)
        m1 = func(x[i], y[i])
        m2 = func(x[i + 1], y[i] + h * m1)
        y = numpy.append(y, y[i] + (h * (m1 + m2)) / 2)
    return x, y

## Runge Kutta Method
Receive function to compute, initial value for x and y, end of the interval [x0,X] and number of steps to perform

Returns x and y arrays

In [8]:
# Get function, initial value for x and y, end of the interval [x,X0] and number of steps to perform
def runge_kutta(func, x0, y0, X, N_steps):
    # Resulting pairs of x, y will be stored in arrays  
    x = get_x(x0, X, N_steps)
    y = numpy.array([y0])
    
    # Calculate the step for x
    h = (X - x0) / float(N_steps)
    
    # Loop until 50 elements (-1 because we start with y0)
    for i in range(N_steps-1):
        # Perform Runge-Kutta method
        k1 = h * func(x[i], y[i])
        k2 = h * func(x[i] + h / 2, y[i] + k1 / 2)
        k3 = h * func(x[i] + h / 2, y[i] + k2 / 2)
        k4 = h * func(x[i] + h, y[i] + k3)
        y = numpy.append(y, y[i] + (k1 + 2 * k2 + 2 * k3 + k4) / 6)
    return x, y

## Computation of all methods
All methods combined in one function since we will compute all of them at the same time a lot

Function returns x and y for every method (exact, euler, improved euler, runge-kutta)

In [9]:
def all_methods(func, x0, y0, X, N_steps):
    x = get_x(x0, X, N_steps)
    # We dont save x as it is the same for everyone so it is saved into '_'
    _, y_exact = exact(x0, y0, X, N_steps) 
    _, y_euler = euler(func, x0, y0, X, N_steps)
    _, y_improved = improved_euler(func, x0, y0, X, N_steps)
    _, y_runge = runge_kutta(func, x0, y0, X, N_steps)
    return x, y_exact, y_euler, y_improved, y_runge

Create widgets for changable x0, y0, X and number of steps

Save default value and result of the methods using them

In [10]:
x0_widget = widgets.FloatSlider(value=0, min=-100, max=100)
y0_widget = widgets.FloatSlider(value=2, min=-100, max=100)
X_widget = widgets.FloatSlider(value=6.4, min=-10, max=8)
N_steps_widget = widgets.IntSlider(value=50, min=45, max=200)

x0=0
y0=2
X=6.4
N_steps=50

# Compute values of x and y for default values
x, y_exact, y_euler, y_improved, y_runge = all_methods(my_func, x0, y0, X, N_steps)

## Comparison of methods via plotting
We will compare different methods by plotting their graphs with the exact solution and absolute error graph

In [11]:
# Create figure
methods_graph = figure(plot_width=400, plot_height=400, title="Methods Comparison", x_axis_label='x', y_axis_label='y')

# Add lines of the method for the figure
exact_line = methods_graph.line(x, y_exact,legend='Exact solution', line_color='#000000')
euler_line = methods_graph.line(x, y_euler,legend='Euler method', line_color='#FF0000', alpha=0.5)
improved_line = methods_graph.line(x, y_improved,legend='Improved Euler method', line_color='#00FF00',alpha=0.5)
runge_line = methods_graph.line(x, y_runge,legend ='Runge-Kutta method', line_color='#0000FF', alpha=0.5)

## Update plot function
Uses jupyter interact to replot graph and error with different steps or/and x0,y0,X

In [12]:
def update_plot(x0=0, y0=2, X=6.4,N_steps=50):
    # X cannot be less or equal to x0
    if X <= x0:
        print("X must be less or equal to x0")
        return
    # y0 != 0 from the solution 
    if y0 == 0:
        print("y0 cannot be equal to 0")
        return
    # Compute all methods
    x, y_exact, y_euler, y_improved, y_runge = all_methods(my_func, x0, y0, X, N_steps)
    
    exact = {'x': x, 'y': y_exact}
    euler = {'x': x, 'y': y_euler}
    improved = {'x': x, 'y': y_improved}
    runge = {'x': x, 'y': y_runge}
    
    # Set new data
    exact_line.data_source.data = exact
    euler_line.data_source.data = euler
    improved_line.data_source.data = improved
    runge_line.data_source.data = runge
    
    # Compute absolute error from exact value
    y_error_euler = numpy.abs(y_exact-y_euler)
    y_error_improved = numpy.abs(y_exact-y_improved)
    y_error_runge = numpy.abs(y_exact-y_runge)
    
    # Create figure, set lines with error and then show it
    error = figure(plot_width=400, plot_height=400, title="Errors", x_axis_label='x', y_axis_label='Absolute error')

    error1_r = error.line(x, y_error_euler, color='red', legend='Euler method', line_width=2)
    error2_r = error.line(x, y_error_improved, color='green', legend='Improved Euler method', line_width=2)
    error3_r = error.line(x, y_error_runge, color='blue', legend='Runge-Kutta method', line_width=2)
    show(error)
    
    # Push new values of methods to plotter
    push_notebook(method_handle)

## Plot of the function
Interactive plot of our function

You can change initial value or number of steps to see how it affects our methods and then see error for them (lower). Though some lines overlap you can increase scale of the plot and see lines in detail

Error in this part shows only absolute difference of every method and exact solution for this particular input
### Analysis of methods and error plot
As we can see, every method has its own error but still it is a close representation of our function. From the graph it is clear that euler method has worser error than improved euler method and improved euler method has worser error than runge-kutta.

We also can note that as we increase number of steps, precision of our method increases and they begin to overlap with our exact solution. This is because our exact solution function as x->infinity equals 0, thus the error will decrease as x increases.


In [13]:
# Show solution of our methods
method_handle = show(methods_graph, notebook_handle=True)
# Interact with initial value and see how it affects our graph
_ = interact(update_plot, x0=x0_widget, y0=y0_widget, X=X_widget,N_steps=N_steps_widget)

interactive(children=(FloatSlider(value=0.0, description='x0', min=-100.0), FloatSlider(value=2.0, description…

## Error graph
Now we will show how the global error changes when we change the number of steps required for our methods using default values

In [14]:
# We will take maximum of 300 steps (from initial 50)
max_steps = 300

# Create figure for global error graph
error_change = figure(plot_width=400, plot_height=400, title="Error change", x_axis_label='Number of steps', y_axis_label='Global Error')
# Create array of steps from 50 to 300 
x_change = list(range(N_steps, max_steps+1))

# We will save global error for every step in arrays
y_euler_global = numpy.array([])
y_improved_global = numpy.array([])
y_runge_global = numpy.array([])

# For every step compute all methods and save the global error for it (step) to our array
for x in x_change:
    _, y1, y2, y3, y4 = all_methods(my_func, x0, y0, X, x)
    y_euler_global = numpy.append(y_euler_global, max(numpy.abs(y1-y2)))
    y_improved_global = numpy.append(y_improved_global, max(numpy.abs(y1-y3)))
    y_runge_global = numpy.append(y_runge_global, max(numpy.abs(y1-y4)))

# Add line for every method and show
error_change.line(x_change, y_euler_global, color='red', legend='Euler method', line_width=2)
error_change.line(x_change, y_improved_global, color='green', legend='Improved Euler method', line_width=2)
error_change.line(x_change, y_runge_global, color='blue', legend='Runge-Kutta method', line_width=2)
show(error_change)

    
    
    
    

### Analysis of global error graph
From the graph we can notice that if we increase our step the global (maximum) error for each method decreases and approaches 0. This is, as was noticed before, because value of our function is approaching 0 (from 2) as x approaches infinity, thus the error decreases as all methods goes to very small numbers.